In [ ]:
import pandas as pd

svm_results = pd.read_csv("svm_results_metrics.csv")
custom_results_cytoplasm = pd.read_csv(
    "results_data/segmentation/cytoplasm/results.csv"
)
custom_results_nucleus = pd.read_csv("results_data/segmentation/nucleis/results.csv")

In [ ]:
custom_results_cytoplasm.describe()

0.0034923544706444705


,image,f1,n_objects
count,50.00000,50.000000,50.000000
mean,25.50000,0.839677,17.580000
std,14.57738,0.059096,5.799683
min,1.00000,0.601793,8.000000
25%,13.25000,0.835446,14.000000
50%,25.50000,0.854901,17.000000
75%,37.75000,0.866509,19.000000
max,50.00000,0.908477,40.000000


In [387]:
custom_results_nucleus.describe()

,image,jaccard,n_objects
count,50.00000,50.000000,50.000000
mean,25.50000,0.685648,25.900000
std,14.57738,0.061397,8.231969
min,1.00000,0.535423,10.000000
25%,13.25000,0.645429,20.000000
50%,25.50000,0.688426,25.000000
75%,37.75000,0.736047,31.000000
max,50.00000,0.779557,45.000000


In [ ]:
svm_results.describe()

,F1_Cytoplasma,F1_Cytoplasma_Mod,Jaccard_Jadra,Jaccard_Jadra_Mod
count,50.000000,50.000000,50.000000,50.000000
mean,0.841313,0.847213,0.698153,0.676707
std,0.048519,0.036879,0.088278,0.091729
min,0.617666,0.683643,0.437467,0.457877
25%,0.833188,0.839865,0.669630,0.623659
50%,0.850956,0.851304,0.715912,0.678821
75%,0.865395,0.870305,0.758544,0.748229
max,0.890838,0.899023,0.825621,0.843992


In [388]:
svm_results["Image"] = svm_results["Image"].apply(lambda x: int(x.split("_")[-1]) + 1)

svm_results.sort_values(by="Image", inplace=True)

In [ ]:
results = svm_results.merge(
    custom_results_cytoplasm,
    left_on="Image",
    right_on="image",
    suffixes=("", "_custom_cytoplasm"),
)
results = results.merge(
    custom_results_nucleus,
    left_on="Image",
    right_on="image",
    suffixes=("", "_custom_nucleus"),
)

In [390]:
results.head()

,Image,F1_Cytoplasma,F1_Cytoplasma_Mod,Jaccard_Jadra,Jaccard_Jadra_Mod,image,f1,time,method,type,image_custom_nucleus,jaccard,n_objects
0,1,0.822739,0.842035,0.710734,0.667941,1,0.893781,28.865014,watershed,cytoplasm,1,0.756817,27
1,2,0.817673,0.837914,0.625872,0.537016,2,0.897278,27.440097,watershed,cytoplasm,2,0.535423,18
2,3,0.864162,0.854131,0.684558,0.570582,3,0.874378,30.943515,watershed,cytoplasm,3,0.545925,22
3,4,0.865549,0.860572,0.693383,0.669668,4,0.784606,29.901289,watershed,cytoplasm,4,0.646705,41
4,5,0.881380,0.874868,0.642882,0.457877,5,0.890388,27.516791,watershed,cytoplasm,5,0.608025,20


In [ ]:
import pandas as pd
import plotly.express as px

melted_results = results.melt(
    value_vars=[
        "F1_Cytoplasma",
        "F1_Cytoplasma_Mod",
        "f1",
        "Jaccard_Jadra",
        "Jaccard_Jadra_Mod",
        "jaccard",
    ],
    var_name="Metric",
    value_name="Value",
)

cytoplasma_results = melted_results[
    melted_results["Metric"].isin(["F1_Cytoplasma", "F1_Cytoplasma_Mod", "f1"])
]
nucleus_results = melted_results[
    melted_results["Metric"].isin(["Jaccard_Jadra", "Jaccard_Jadra_Mod", "jaccard"])
]

In [393]:
fig = px.box(
    cytoplasma_results,
    x="Metric",
    y="Value",
    points="all",
    color="Metric",
    template="plotly_white",
)

fig.update_layout(
    showlegend=False,
    width=1600,
    height=600,
    xaxis_title=None,
    yaxis_title="F1",
    font=dict(size=16),
)

fig.update_xaxes(
    ticktext=[
        "SVM",
        "SVM mod",
        "Vlastní algoritmus",
    ],
    tickvals=[
        "F1_Cytoplasma",
        "F1_Cytoplasma_Mod",
        "f1",
    ],
)

fig.show()

In [395]:
fig = px.box(
    nucleus_results,
    x="Metric",
    y="Value",
    points="all",
    color="Metric",
    template="plotly_white",
)

fig.update_layout(
    showlegend=False,
    width=1600,
    height=600,
    xaxis_title=None,
    yaxis_title="IoU",
    font=dict(size=16),
)

fig.update_xaxes(
    ticktext=[
        "SVM",
        "SVM mod",
        "Vlastní algoritmus",
    ],
    tickvals=["Jaccard_Jadra", "Jaccard_Jadra_Mod", "jaccard"],
)


fig.show()